In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),".."))

import model_utils

import polars as pl
# import data
def load_data():
    balltoball = pl.read_csv(os.path.join(os.path.join( '..',"data", "filtered_data" , "balltoball.csv")))
    team_stats = pl.read_csv(os.path.join(os.path.join( '..',"data", "filtered_data" , "team12_stats.csv")))
    players_stats = pl.read_csv(os.path.join(os.path.join( '..',"data", "filtered_data" , "players_stats.csv")))
    return balltoball, team_stats, players_stats
balltoball,team_stats,players_stats = load_data()
print(balltoball.columns)
print(team_stats.columns)
print(players_stats.columns)
print(balltoball.head(1),team_stats.head(1),players_stats.head(1))

['match_id', 'flip', 'innings', 'ball', 'runs', 'wickets', 'overs', 'run_rate', 'curr_score', 'curr_wickets', 'target', 'won']
['match_id', 'flip', 'gender', 'Cumulative Won team1', 'Cumulative Lost team1', 'Cumulative Tied team1', 'Cumulative W/L team1', 'Cumulative AveRPW team1', 'Cumulative AveRPO team1', 'Cumulative Won team2', 'Cumulative Lost team2', 'Cumulative Tied team2', 'Cumulative W/L team2', 'Cumulative AveRPW team2', 'Cumulative AveRPO team2']
['match_id', 'flip', 'Cum Mat Total', 'Cum Runs Total', 'Cum SR', 'Cumulative Overs', 'Cumulative Bowling Runs', 'Cumulative Wkts', 'Cumulative Econ', 'Cumulative Dis', 'Cumulative Ct', 'Cumulative St', 'Cumulative D/I']
shape: (1, 12)
┌──────────┬──────┬─────────┬──────┬───┬────────────┬──────────────┬────────┬─────┐
│ match_id ┆ flip ┆ innings ┆ ball ┆ … ┆ curr_score ┆ curr_wickets ┆ target ┆ won │
│ ---      ┆ ---  ┆ ---     ┆ ---  ┆   ┆ ---        ┆ ---          ┆ ---    ┆ --- │
│ i64      ┆ i64  ┆ i64     ┆ f64  ┆   ┆ i64      

In [2]:
def partition_data(df, group_keys):
    partitions = df.partition_by(group_keys)
    partition_list = [partition.drop(group_keys).to_numpy() for partition in partitions]
    return partition_list

balltoball_partitions = partition_data(balltoball, ["match_id","flip"])
team_stats_partitions = partition_data(team_stats, ["match_id","flip"])
players_stats_partitions = partition_data(players_stats, ["match_id","flip"])

import numpy as np
labels = np.array([])
for i in range(0, len(balltoball_partitions)):
    labels = np.append(labels, balltoball_partitions[i][:,-1][0])
    balltoball_partitions[i] = balltoball_partitions[i][:,:-1]

In [3]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

dataset = model_utils.CricketDataset(team_stats_partitions, players_stats_partitions, balltoball_partitions, labels)

train_indices, temp_indices = train_test_split(
    list(range(len(dataset))), stratify=labels, test_size=0.3, random_state=42)

val_indices, test_indices = train_test_split(
    temp_indices, stratify=[labels[i] for i in temp_indices], test_size=0.5, random_state=42)

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=model_utils.collate_fn_with_padding)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, collate_fn=model_utils.collate_fn_with_padding)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=model_utils.collate_fn_with_padding)

for team_input, player_input, ball_input, labels, mask in train_dataloader:
    print(f"Team input shape: {team_input.shape}")  # [batch_size, team_feature_dim]
    print(f"Player input shape: {player_input.shape}")  # [batch_size, player_feature_dim]
    print(f"Padded ball input shape: {ball_input.shape}")  # [batch_size, max_seq_len, ball_feature_dim]
    print(f"Mask shape: {mask.shape}")  # [batch_size, max_seq_len]
    print(f"Labels shape: {labels.shape}")  # [batch_size]
    break

Team input shape: torch.Size([32, 13])
Player input shape: torch.Size([32, 22, 11])
Padded ball input shape: torch.Size([32, 255, 9])
Mask shape: torch.Size([32, 255])
Labels shape: torch.Size([32])


In [4]:
import pickle
with open(os.path.join( '..',"data", "pytorch_data" , "train_dataloader.pkl"), "wb") as f:
    pickle.dump(train_dataloader, f)

with open(os.path.join( '..',"data", "pytorch_data" , "val_dataloader.pkl"), "wb") as f:
    pickle.dump(val_dataloader, f)

with open(os.path.join( '..',"data", "pytorch_data" , "test_dataloader.pkl"), "wb") as f:
    pickle.dump(test_dataloader, f)

In [5]:
train_dataloader = pickle.load(open(os.path.join( '..',"data", "pytorch_data" , "train_dataloader.pkl"), "rb"))
val_dataloader = pickle.load(open(os.path.join( '..',"data", "pytorch_data" , "val_dataloader.pkl"), "rb"))
test_dataloader = pickle.load(open(os.path.join( '..',"data", "pytorch_data" , "test_dataloader.pkl"), "rb"))

In [13]:
for i in train_dataloader:
    data0 = np.array(i[0][0]).reshape(1,-1)
    data1 = np.array(i[1][0])
    data2 = np.array(i[2][0])
    print(data0.shape, data1.shape, data2.shape)
    team0 = pl.DataFrame(data0,schema=team_stats.columns[2:])
    players1 = pl.DataFrame(data1,schema=players_stats.columns[2:])
    balltoball2 = pl.DataFrame(data2,schema=balltoball.columns[2:-1])


    break
team0

(1, 13) (22, 11) (255, 9)


gender,Cumulative Won team1,Cumulative Lost team1,Cumulative Tied team1,Cumulative W/L team1,Cumulative AveRPW team1,Cumulative AveRPO team1,Cumulative Won team2,Cumulative Lost team2,Cumulative Tied team2,Cumulative W/L team2,Cumulative AveRPW team2,Cumulative AveRPO team2
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0.0,91.0,76.0,3.0,1.2,27.43,8.36,95.0,69.0,1.0,1.38,28.190001,8.33


In [14]:
players1

Cum Mat Total,Cum Runs Total,Cum SR,Cumulative Overs,Cumulative Bowling Runs,Cumulative Wkts,Cumulative Econ,Cumulative Dis,Cumulative Ct,Cumulative St,Cumulative D/I
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17.0,345.0,136.199997,6.0,56.0,1.0,9.88,4.0,4.0,0.0,0.24
46.0,1086.0,127.639999,44.0,348.0,15.0,7.84,20.0,20.0,0.0,0.43
9.0,220.0,138.429993,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.44
51.0,803.0,142.580002,71.199997,614.0,18.0,8.87,16.0,16.0,0.0,0.32
…,…,…,…,…,…,…,…,…,…,…
2.0,12.0,70.580002,8.0,90.0,4.0,11.25,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6.0,4.0,57.139999,23.4,208.0,9.0,8.78,2.0,2.0,0.0,0.33


In [15]:
balltoball2

innings,ball,runs,wickets,overs,run_rate,curr_score,curr_wickets,target
f32,f32,f32,f32,f32,f32,f32,f32,f32
1.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0,0.3,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1.0,0.4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1.0,0.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0
…,…,…,…,…,…,…,…,…
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Define the models
class TeamStatsModel(nn.Module):
    def __init__(self, input_size):
        super(TeamStatsModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.5),
            nn.Linear(32, 16),
            nn.ReLU()
        )

    def forward(self, x):
        return self.model(x)

class PlayerStatsModel(nn.Module):
    def __init__(self, input_size, seq_len):
        super(PlayerStatsModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=32, kernel_size=3)
        self.bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * ((seq_len - 4) // 4), 16)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = self.flatten(x)
        x = F.relu(self.fc(x))
        return x

class BallToBallModel(nn.Module):
    def __init__(self, input_dim):
        super(BallToBallModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, 128, num_layers=2, batch_first=True, bidirectional=False)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(128, 16)

    def forward(self, x, lengths):
        x_packed = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        output_packed, (hn, cn) = self.lstm(x_packed)
        hn = hn[-1,:,:]
        x = self.dropout(hn)
        x = F.relu(self.fc(x))
        return x

class CombinedModel(nn.Module):
    def __init__(self, team_input_size, player_input_size, player_seq_len, ball_input_dim):
        super(CombinedModel, self).__init__()
        self.team_model = TeamStatsModel(team_input_size)
        self.player_model = PlayerStatsModel(player_input_size, player_seq_len)
        self.ball_model = BallToBallModel(ball_input_dim)
        self.fc = nn.Sequential(
            nn.Linear(16+16+16, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, 1)
        )

    def forward(self, team_input, player_input, ball_input, ball_lengths):
        team_output = self.team_model(team_input)
        player_output = self.player_model(player_input)
        ball_output = self.ball_model(ball_input, ball_lengths)
        combined = torch.cat((team_output, player_output, ball_output), dim=1)
        output = self.fc(combined)
        return output.squeeze()

# Function to print a sample of the training and validation data
def print_data_sample(train_dataloader, val_dataloader):
    print("Sample from training data:")
    for i, (team_input, player_input, ball_input, labels, ball_lengths) in enumerate(train_dataloader):
        if i >= 1:  # Print only the first batch
            break
        print(f"Team input: {team_input[0]}")
        print(f"Player input: {player_input[0]}")
        print(f"Ball input: {ball_input[0]}")
        print(f"Label: {labels[0]}")
        print(f"Ball lengths: {ball_lengths[0]}")
    
    print("\nSample from validation data:")
    for i, (team_input, player_input, ball_input, labels, ball_lengths) in enumerate(val_dataloader):
        if i >= 1:  # Print only the first batch
            break
        print(f"Team input: {team_input[0]}")
        print(f"Player input: {player_input[0]}")
        print(f"Ball input: {ball_input[0]}")
        print(f"Label: {labels[0]}")
        print(f"Ball lengths: {ball_lengths[0]}")

In [37]:
config = load_config()
device = get_device()

# Load data
teamStats, playersStats, balltoball = load_data(config['data_directory'])

# Preprocess data
team_stats_partitions = partition_data(teamStats, ['match_id', 'flip'])
player_stats_partitions = partition_data(playersStats, ['match_id', 'flip'])
ball_stats_partitions = partition_data(balltoball, ['match_id', 'flip'])

# Apply data augmentation
augmented_team_stats, augmented_player_stats, augmented_ball_stats, augmented_labels = augment_data(
    team_stats_partitions, player_stats_partitions, ball_stats_partitions)

# Convert labels to integers
augmented_labels = [int(label) for label in augmented_labels]

2024-11-08 09:32:41,172 - INFO - Using device: cuda


In [38]:
print_data_sample(train_dataloader, val_dataloader)

Sample from training data:
Team input: tensor([ 0.0000, 70.0000, 88.0000,  3.0000, 10.0000,  0.8000, 23.7100,  7.9100,
        91.0000, 63.0000,  1.0000,  2.0000,  1.4400, 28.0300,  8.2300])
Player input: tensor([[3.6000e+01, 3.2000e+01, 6.5100e+02, 2.4200e+01, 1.4130e+02, 1.0000e+00,
         1.4000e+01, 0.0000e+00, 1.4000e+01, 2.7000e+01, 2.4000e+01, 3.0000e+00,
         7.5000e-01],
        [6.9000e+01, 6.8000e+01, 1.8940e+03, 3.2070e+01, 1.2744e+02, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 7.7000e+01, 6.2000e+01, 1.5000e+01,
         1.1200e+00],
        [4.5000e+01, 1.6000e+01, 1.3900e+02, 2.2000e+01, 1.1197e+02, 1.4330e+02,
         1.1910e+03, 5.0000e+01, 8.4700e+00, 6.0000e+00, 6.0000e+00, 0.0000e+00,
         1.3000e-01],
        [2.3000e+01, 2.1000e+01, 7.2200e+02, 4.4660e+01, 1.4963e+02, 1.9000e+01,
         1.3900e+02, 6.0000e+00, 7.1800e+00, 2.1000e+01, 2.1000e+01, 0.0000e+00,
         9.1000e-01],
        [1.3000e+01, 6.0000e+00, 4.0000e+01, 1.1170e+01, 8.